In [3]:
import numpy as np

In [4]:
np.random.seed(0)

In [5]:
m, n, p = 2, 3, 4

In [6]:
def generate_quantization_constants(alpha, beta, alpha_q, beta_q):
    s = (beta - alpha)/(beta_q - alpha_q)
    z = int((beta * alpha_q - alpha * beta_q)/(beta - alpha))
    
    return s, z

In [7]:
def generate_quantization_int8_constants(alpha, beta):
    b = 8
    alpha_q = -2**(b - 1)
    beta_q = 2**(b - 1) - 1
    
    s, z = generate_quantization_constants(alpha, beta, alpha_q, beta_q)
    
    return s, z

In [11]:
def quantization(x, s, z, alpha_q, beta_q):
    x_q = np.round(1/s*x + z, decimals=0)
    x_q = np.clip(x_q, a_min=alpha_q, a_max=beta_q)
    
    return x_q

In [14]:
def quantization_int8(x, s, z):
    x_q = quantization(x, s, z, -128, 127)
    
    x_q = x_q.astype(np.int8)
    return x_q

In [15]:
alpha_X = -100.0
beta_X = 80.0
s_X, z_X = generate_quantization_int8_constants(alpha_X, beta_X)
X = np.random.uniform(low=alpha_X, high=beta_X, size=(m, p)).astype(np.float32)
X_q = quantization_int8(X, s_X, z_X)

In [16]:
X, X_q

(array([[-96.36069 ,  49.87157 ,  40.068214,  56.60219 ],
        [ 76.1513  ,  43.84854 , -16.933714,  40.49525 ]], dtype=float32),
 array([[-124,   84,   70,   93],
        [ 121,   75,  -11,   70]], dtype=int8))

In [17]:
alpha_W = -20.0
beta_W = 10.0
s_W, z_W = generate_quantization_int8_constants(alpha=alpha_W, beta=beta_W)
W = np.random.uniform(low=alpha_W, high=beta_W, size=(p, n)).astype(np.float32)
W_q = quantization_int8(x=W, s=s_W, z=z_W)

In [18]:
alpha_b = -500.0
beta_b = 500.0
s_b, z_b = generate_quantization_int8_constants(alpha=alpha_b, beta=beta_b)
b = np.random.uniform(low=alpha_b, high=beta_b, size=(1, n)).astype(np.float32)
b_q = quantization_int8(x=b, s=s_b, z=z_b)

In [19]:
# Y
alpha_Y = -3000.0
beta_Y = 3000.0
s_Y, z_Y = generate_quantization_int8_constants(alpha=alpha_Y, beta=beta_Y)
Y_expected = np.matmul(X, W) + b
Y_q_expected = quantization_int8(x=Y_expected, s=s_Y, z=z_Y)

In [20]:
def quantization_matrix_multiplication_int8(X_q, W_q, b_q, s_X, z_X, s_W, z_W, s_b, z_b, s_Y, z_Y):

    p = W_q.shape[0]

    Y_q_simulated = (z_Y + (s_b / s_Y * (b_q.astype(np.int32) - z_b)).astype(np.int8) + ((s_X * s_W / s_Y) * (np.matmul(X_q.astype(np.int32), W_q.astype(np.int32)) - z_W * np.sum(X_q.astype(np.int32), axis=1, keepdims=True) - z_X * np.sum(W_q.astype(np.int32), axis=0, keepdims=True) + p * z_X * z_W)).astype(np.int8)).astype(np.int8)

    return Y_q_simulated

In [21]:
Y_q_simulated = quantization_matrix_multiplication_int8(X_q=X_q, W_q=W_q, b_q=b_q, 
                                                        s_X=s_X, z_X=z_X, s_W=s_W, z_W=z_W, 
                                                        s_b=s_b, z_b=z_b, s_Y=s_Y, z_Y=z_Y)

In [24]:
def dequantization(x, s, z):
    x = s * (x - z)
    x = x.astype(np.float32)
    
    return x

In [25]:
Y_expected, dequantization(Y_q_simulated, s_Y, z_Y)

(array([[ 1463.1698 ,  -993.25555,  1243.2084 ],
        [ -690.0896 ,  -976.393  , -1035.904  ]], dtype=float32),
 array([[ 1435.2941,  -988.2353,  1200.    ],
        [ -705.8823,  -964.7059, -1035.2941]], dtype=float32))